In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

import MCCoordinator
import StorageSampler

def storage_system(**kwargs):
    """
    Instantiate StorageSystem object using GB system data

    :param wind_power: assumed wind power capacity (in MW)
    :param kwargs: additional arguments to be supplied to the StorageSystem constructor
    :return: StorageSystem object
    """
    import pandas as pd
    import os
    data = pd.read_csv('../data/UKdata/20161213_uk_wind_solar_demand_temperature.csv',
                       parse_dates=['UTC Time', 'Local Time'], infer_datetime_format=True, dayfirst=True, index_col=0)

    demand_data = data['demand_net'].dropna()['2006':'2015']
    wind_data = 10000 * data['wind_merra1'].dropna()

    demand_samples = {yeardata[0]: yeardata[1].values[:8760] for yeardata in
                      demand_data.groupby(demand_data.index.year)}
    wind_samples = {yeardata[0]: yeardata[1].values[:8760] for yeardata in wind_data.groupby(wind_data.index.year)}

    dataframe = pd.read_csv('../data/UKdata/battery_data.csv')
    store_power_list=3*dataframe['Power (MW)'][0:27]
    store_energy_list=3*dataframe['Energy (MWh)'][0:27]

    return StorageSampler.StorageSystem(demand_samples=demand_samples, 
                                        wind_samples=wind_samples, 
                                        store_power_list=store_power_list,
                                        store_energy_list=store_energy_list,
                                        **kwargs)


def pool_generation(system, pool_size):
    pool_temporal = np.zeros((pool_size, 24))

    for i in range(pool_size):
        pool_temporal[i] = system.generate_daily_margin_trace()
    return pool_temporal

def load_data(file_name):
    '''
    load data from the file_name
    Parameters: 
        file_name: string
            file address
    Returns:
        data: ndarray
            an array of file's content
    '''
    data = np.ascontiguousarray(np.genfromtxt(file_name, delimiter=','))
    return data


system = storage_system()

import MachineLearning
system.AI_model = MachineLearning.MachineLearning(train_size=500)

/var/folders/pr/b8ltbq5x39n0gw92v8zlc8q00000gn/T/ipykernel_74696/187439617.py:19: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data = pd.read_csv('../data/UKdata/20161213_uk_wind_solar_demand_temperature.csv',


Generator set: [1200, 600, 600, 250, 250, 120, 60, 20, 20, 10, 10, 10]
Optimising number of generator sets to LOLE target of 3 hours
20 generator sets. Base LOLE=2.92 h; determined load offset of 36.15 MW to reach LOLE target.



In [ ]:
###################################################################################################
# Test set (Daily 24 hours) for calculating the RMSE of the surrogate model
start_time = time.time()

test_daily_size = 100000 # Number of days to generate

# Generate daily test data
X_test_daily = pool_generation(system, test_daily_size)

end_time = time.time()
print('Time to generate daily test data: ', end_time - start_time)

# Label the daily test data
start_time = time.time()

y_test_daily = np.array([system.run_optimal_policy(X_test_daily[i]) for i in range(X_test_daily.shape[0])])
lol_test_daily = y_test_daily[:,0]
ens_test_daily = y_test_daily[:,1]

end_time = time.time()
print('Time to label the daily test data: ', end_time - start_time)

# Save testing data
# np.savetxt("daily_margin_test.csv", X_test_daily, delimiter=",")
# np.savetxt("lol_test_daily.csv", lol_test_daily, delimiter=",")
# np.savetxt("ens_test_daily.csv", ens_test_daily, delimiter=",")

##################################################################################################
# Test set (Yearly 8760 hours) for calculating the correlation between the exact and surrogate models
start_time = time.time()

test_yearly_size = 1000 # Number of years to generate

# Generate the yearly test data
X_test_yearly = np.zeros((test_yearly_size, 8760))

for i in range(test_yearly_size):
    X_test_yearly[i] = system.generate_margin_trace()

end_time = time.time()
print('Time to generate test-year data: ', end_time - start_time)

# Label training data
start_time = time.time()

y_test_yearly = np.array([system.run_optimal_policy(X_test_yearly[i]) for i in range(X_test_yearly.shape[0])])
lol_test_yearly = y_test_yearly[:,0]
ens_test_yearly = y_test_yearly[:,1]

end_time = time.time()
print('Time to label the yearly test labels: ', end_time - start_time)

# Save testing data
# np.savetxt("yearly_margin_test.csv", X_test_yearly, delimiter=",")
# np.savetxt("lol_test_yearly.csv", lol_test_yearly, delimiter=",")
# np.savetxt("ens_test_yearly.csv", ens_test_yearly, delimiter=",")

Time to generate test-year data:  23.53668713569641
Time to label the yearly test labels:  12812.81214928627


In [ ]:
# Loading the test set
X_test = load_data("../data/AIdata/daily_margin_test.csv")
ens_test = load_data("../data/AIdata/ens_test_daily.csv")
lol_test = load_data("../data/AIdata/lol_test_daily.csv")

# # Analyzing the test set
# count = 0
# for i in range(ens_test.shape[0]):
#     if ens_test[i] > 0:
#         count += 1
# print("Fraction of ens days: ", count/ens_test.shape[0])


# print("Total LOLP: ", np.sum(lol_test)/(lol_test.shape[0]*24))
# print("lol_test.shape[0]: ", lol_test.shape[0])


# Load the yearly test set for calculating the correlation
X_test_yearly = load_data("../data/AIdata/yearly_margin_test.csv")
ens_test_yearly = load_data("../data/AIdata/ens_test_yearly.csv")
lol_test_yearly = load_data("../data/AIdata/lol_test_yearly.csv")